In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [3]:
def prepare_data(features_df):
    X = features_df.drop(columns=['Output'])
    y = features_df['Output']
    y = y.map({'legitimate': 0, 'dga': 1})
    return X, y

In [5]:
def shuffle_data(X, y):
    combined = pd.concat([X, y], axis=1)
    shuffled = combined.sample(frac=1, random_state=42).reset_index(drop=True)
    X = shuffled.drop(columns=['Output'])
    y = shuffled['Output']
    return X, y

In [7]:
def train_classifier(X, y, classifier, params=None):
    X, y = shuffle_data(X, y)
    print( y.value_counts())
    #to handle class imbalance
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    
    
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)
    
    # Use GridSearchCV for hyperparameter tuning 
    if params:
        clf = GridSearchCV(classifier, params, cv=5, scoring='accuracy')
    else:
        clf = classifier
    
    # Train the model
    clf.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = clf.predict(X_test)
    print("Best Parameters:", clf.best_params_ if params else "No Grid Search")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

    return clf

In [9]:
def main():
    features_df = pd.read_csv('features.csv')
    X, y = prepare_data(features_df)
    
    # Try Decision Tree
    print("Decision Tree Classifier:")
    dt_params = {'max_depth': [5, 10, 15], 
                 'min_samples_split': [10, 15, 20], 
                 'min_samples_leaf': [1, 5, 10]}
    dt_clf = train_classifier(X, y, DecisionTreeClassifier(random_state=42), dt_params)
    
    # Try Random Forest
    print("\nRandom Forest Classifier:")
    rf_params = {'n_estimators': [50, 100, 200], 
                 'max_depth': [5, 10, 15], 
                 'min_samples_split': [10, 15, 20]}
    rf_clf = train_classifier(X, y, RandomForestClassifier(random_state=42), rf_params)
    
    # Try SVM
    print("\nSVM Classifier:")
    svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    svm_clf = train_classifier(X, y, SVC(probability=True, random_state=42), svm_params)

if __name__ == "__main__":
    main()

Decision Tree Classifier:
Output
1    27911
0    26160
Name: count, dtype: int64
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
Accuracy: 0.9999402878127426
Confusion Matrix:
 [[8424    1]
 [   0 8322]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8425
           1       1.00      1.00      1.00      8322

    accuracy                           1.00     16747
   macro avg       1.00      1.00      1.00     16747
weighted avg       1.00      1.00      1.00     16747

ROC AUC Score: 0.999940652818991

Random Forest Classifier:
Output
1    27911
0    26160
Name: count, dtype: int64
Best Parameters: {'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 50}
Accuracy: 1.0
Confusion Matrix:
 [[8425    0]
 [   0 8322]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8425
           1       1.00 